# Single-Node EKS Cluster Setup

This notebook guides you through the process of creating a single-node EKS (Elastic Kubernetes Service) cluster on AWS and configuring kubectl to manage it.

## Setup aws cli

My company uses sso, so running `aws configure sso` would e appropriate. You can use whichever method works best so long as the user is able to create EKS clusters, IAM Rols, VPCs and Node Groups.

## Set Variables

First, let's set the variables we'll use throughout this process:

In [ ]:
# Set variables
export CLUSTER_NAME="my-single-node-cluster"
export REGION="us-east-1"  # Change this to your preferred region
export NODE_GROUP_NAME="single-node-group"
export NODE_INSTANCE_TYPE="t3.medium"  # Smallest recommended size for EKS
export NODE_COUNT=1


## Step 1: Create IAM Role for EKS Cluster

Create an IAM role that the EKS service can assume to manage cluster resources:

In [ ]:
echo "Creating IAM role for EKS cluster..."
aws iam create-role \
  --role-name EKSClusterRole \
  --assume-role-policy-document '{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": "eks.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  }'

# Attach required policies to the role
aws iam attach-role-policy \
  --role-name EKSClusterRole \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKSClusterPolicy

## Step 2: Create IAM Role for EKS Node Group

Create an IAM role for the worker nodes that will run in the EKS cluster:

In [ ]:
echo "Creating IAM role for EKS node group..."
aws iam create-role \
  --role-name EKSNodeRole \
  --assume-role-policy-document '{
    "Version": "2012-10-17",
    "Statement": [
      {
        "Effect": "Allow",
        "Principal": {
          "Service": "ec2.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
      }
    ]
  }'

# Attach required policies to the node role
aws iam attach-role-policy \
  --role-name EKSNodeRole \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKSWorkerNodePolicy

aws iam attach-role-policy \
  --role-name EKSNodeRole \
  --policy-arn arn:aws:iam::aws:policy/AmazonEKS_CNI_Policy

aws iam attach-role-policy \
  --role-name EKSNodeRole \
  --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly

## Step 3: Create VPC for EKS

Create a VPC using AWS CloudFormation template specifically designed for EKS:

In [ ]:
echo "Creating VPC for EKS..."
aws cloudformation create-stack \
  --stack-name eks-vpc \
  --template-url https://s3.us-west-2.amazonaws.com/amazon-eks/cloudformation/2020-10-29/amazon-eks-vpc-private-subnets.yaml

# Wait for the VPC creation to complete
echo "Waiting for VPC creation to complete..."
aws cloudformation wait stack-create-complete --stack-name eks-vpc

## Get VPC and Subnet Information

Retrieve the VPC and subnet IDs created by CloudFormation and create a security group:

In [ ]:
# Get VPC and subnet information
VPC_ID=$(aws cloudformation describe-stacks --stack-name eks-vpc --query "Stacks[0].Outputs[?OutputKey=='VpcId'].OutputValue" --output text)
SUBNET_IDS=$(aws cloudformation describe-stacks --stack-name eks-vpc --query "Stacks[0].Outputs[?OutputKey=='SubnetIds'].OutputValue" --output text)
SECURITY_GROUP_ID=$(aws ec2 create-security-group --group-name eks-cluster-sg --description "Security group for EKS cluster" --vpc-id $VPC_ID --output text --query "GroupId")

# Allow inbound traffic within the security group
aws ec2 authorize-security-group-ingress \
  --group-id $SECURITY_GROUP_ID \
  --protocol all \
  --source-group $SECURITY_GROUP_ID

echo "VPC ID: $VPC_ID"
echo "Subnet IDs: $SUBNET_IDS"
echo "Security Group ID: $SECURITY_GROUP_ID"

## Step 4: Create EKS Cluster

Create the EKS cluster using the IAM role and networking resources we've set up:

In [ ]:
echo "Creating EKS cluster ${CLUSTER_NAME}..."
aws eks create-cluster \
  --name ${CLUSTER_NAME} \
  --role-arn $(aws iam get-role --role-name EKSClusterRole --query "Role.Arn" --output text) \
  --resources-vpc-config subnetIds=${SUBNET_IDS},securityGroupIds=${SECURITY_GROUP_ID}

# Wait for the cluster to be active
echo "Waiting for EKS cluster to become active (this might take 10-15 minutes)..."
aws eks wait cluster-active --name ${CLUSTER_NAME} --region ${REGION}

## Step 5: Create a Node Group with a Single Node

Create a node group that will manage our single worker node:

In [ ]:
echo "Creating node group with a single node..."
aws eks create-nodegroup \
  --cluster-name ${CLUSTER_NAME} \
  --nodegroup-name ${NODE_GROUP_NAME} \
  --node-role $(aws iam get-role --role-name EKSNodeRole --query "Role.Arn" --output text) \
  --subnets $(echo $SUBNET_IDS | tr ',' ' ') \
  --scaling-config minSize=1,maxSize=1,desiredSize=1 \
  --instance-types ${NODE_INSTANCE_TYPE}

# Wait for the node group to be active
echo "Waiting for node group to become active (this might take 5-10 minutes)..."
aws eks wait nodegroup-active \
  --cluster-name ${CLUSTER_NAME} \
  --nodegroup-name ${NODE_GROUP_NAME} \
  --region ${REGION}

## Step 6: Configure kubectl

Update the kubectl configuration to connect to the new EKS cluster:

In [ ]:
# Update kubeconfig to set the context to the new cluster
echo "Updating kubeconfig to set context to the new cluster..."
aws eks update-kubeconfig --name ${CLUSTER_NAME} --region ${REGION}

## Verify Cluster Connection

Verify that kubectl can connect to the cluster and retrieve node information:

In [ ]:
# Verify the cluster connection
echo "Verifying connection to the cluster..."
kubectl get nodes

In [ ]:
echo "EKS single-node cluster setup complete. Your kubectl context has been set to ${CLUSTER_NAME}"
echo "Current context: $(kubectl config current-context)"

## Cleanup (Optional)

When you're done with the cluster, you can use the following commands to clean up resources and avoid ongoing charges:

In [ ]:
# Delete the node group
# aws eks delete-nodegroup --cluster-name ${CLUSTER_NAME} --nodegroup-name ${NODE_GROUP_NAME} --region ${REGION}
# echo "Waiting for node group deletion..."
# aws eks wait nodegroup-deleted --cluster-name ${CLUSTER_NAME} --nodegroup-name ${NODE_GROUP_NAME} --region ${REGION}

# Delete the cluster
# aws eks delete-cluster --name ${CLUSTER_NAME} --region ${REGION}
# echo "Waiting for cluster deletion..."
# aws eks wait cluster-deleted --name ${CLUSTER_NAME} --region ${REGION}

# Delete the security group
# aws ec2 delete-security-group --group-id ${SECURITY_GROUP_ID}

# Delete the VPC CloudFormation stack
# aws cloudformation delete-stack --stack-name eks-vpc

# Delete IAM roles and policies
# aws iam detach-role-policy --role-name EKSNodeRole --policy-arn arn:aws:iam::aws:policy/AmazonEKSWorkerNodePolicy
# aws iam detach-role-policy --role-name EKSNodeRole --policy-arn arn:aws:iam::aws:policy/AmazonEKS_CNI_Policy
# aws iam detach-role-policy --role-name EKSNodeRole --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly
# aws iam delete-role --role-name EKSNodeRole

# aws iam detach-role-policy --role-name EKSClusterRole --policy-arn arn:aws:iam::aws:policy/AmazonEKSClusterPolicy
# aws iam delete-role --role-name EKSClusterRole